## Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.0.0-beta1
2.2.4-tf


In [2]:
from tensorflow.keras.layers import ReLU,Dense, BatchNormalization, Softmax, GlobalAveragePooling2D

In [3]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [4]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

## Hyper Parameters

In [5]:
learning_rate = 0.0001
training_epochs = 2
batch_size = 32
img_size = 224

## TensorFlow_Datasets

In [6]:
## train, validation, test 비율 설정
SPLIT_WEIGHTS = (8, 1, 1)
## cats vs dogs dataset을 tensorflow_datasets에서 load하고
## training dataset을 위에서 설정한 비율대로 나누어서 train, validation, test dataset으로 구성
splits = tfds.Split.TRAIN.subsplit(weighted=SPLIT_WEIGHTS)
(raw_train, raw_validation, raw_test), metadata = tfds.load('cats_vs_dogs', split=list(splits),
                                                            with_info=True, as_supervised=True)

In [7]:
print(raw_train)
print(raw_validation)
print(raw_test)

<_OptionsDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>
<_OptionsDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>
<_OptionsDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>


In [8]:
## Preprocessing 함수
@tf.function
def format_example(image, label):
    image = tf.cast(image, tf.float32)
    image = image/255.
    image = tf.image.resize(image, (img_size, img_size))
    label = tf.one_hot(label, 2)
    return image, label

In [9]:
## Preprecessing 함수를 dataset에 적용
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [10]:
## image와 label 확인
for images, labels in train.take(2):
    print(images.shape)
    print(labels.numpy())    

(224, 224, 3)
[0. 1.]
(224, 224, 3)
[1. 0.]


In [11]:
## batch 및 shuffle 적용
train_dataset = train.shuffle(1000).batch(batch_size).repeat()
val_dataset = validation.batch(batch_size)
test_dataset = test.batch(batch_size)

In [12]:
## batch size 반영되어 있는지 확인
for images, labels in train_dataset.take(1):
    print(images.shape)
    print(labels.shape)

(32, 224, 224, 3)
(32, 2)


## Loading Pretrained Model

In [13]:
## Pretrained model load
conv_base = MobileNetV2(weights='imagenet', include_top=False,
                       input_shape=(img_size, img_size, 3))

In [14]:
## Dense layer 추가하여 network 구성
def create_model():
    model = keras.models.Sequential()
    model.add(conv_base)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dense(2))
    model.add(BatchNormalization())
    model.add(Softmax())
    return model

In [15]:
model = create_model()

In [16]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate),
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               163968    
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
re_lu (ReLU)                 (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
_________________________________________________________________
batch_normalization_1 (Batch (None, 2)                 8

In [17]:
## Training, Validation, Test data 수 확인
num_train, num_val, num_test = (
  metadata.splits['train'].num_examples*weight/10
  for weight in SPLIT_WEIGHTS
)
print(num_train, num_val, num_test)

18609.6 2326.2 2326.2


In [18]:
## steps_per_epoch, validation_steps 계산
steps_per_epoch = round(num_train)//batch_size
validation_steps = round(num_val)//batch_size
print(steps_per_epoch, validation_steps)

581 72


In [19]:
## initial loss & accuracy 계산
loss0, accuracy0 = model.evaluate(val_dataset, steps=validation_steps)

72/72 [==============================] - ETA: 3:01 - loss: 0.8880 - accuracy: 0.31 - ETA: 1:00 - loss: 0.8400 - accuracy: 0.41 - ETA: 36s - loss: 0.8123 - accuracy: 0.4563 - ETA: 26s - loss: 0.7949 - accuracy: 0.482 - ETA: 20s - loss: 0.7865 - accuracy: 0.493 - ETA: 16s - loss: 0.7638 - accuracy: 0.511 - ETA: 15s - loss: 0.7752 - accuracy: 0.510 - ETA: 12s - loss: 0.7781 - accuracy: 0.513 - ETA: 11s - loss: 0.7896 - accuracy: 0.502 - ETA: 10s - loss: 0.7910 - accuracy: 0.498 - ETA: 9s - loss: 0.7894 - accuracy: 0.501 - ETA: 8s - loss: 0.7928 - accuracy: 0.49 - ETA: 7s - loss: 0.7892 - accuracy: 0.50 - ETA: 7s - loss: 0.7877 - accuracy: 0.50 - ETA: 6s - loss: 0.7866 - accuracy: 0.50 - ETA: 5s - loss: 0.7894 - accuracy: 0.50 - ETA: 5s - loss: 0.7894 - accuracy: 0.50 - ETA: 5s - loss: 0.7903 - accuracy: 0.50 - ETA: 4s - loss: 0.7915 - accuracy: 0.49 - ETA: 4s - loss: 0.7952 - accuracy: 0.49 - ETA: 4s - loss: 0.7898 - accuracy: 0.50 - ETA: 3s - loss: 0.7887 - accuracy: 0.50 - ETA: 3s - los

In [20]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 0.82
initial accuracy: 0.49


## Training

In [21]:
history = model.fit(train_dataset, epochs=training_epochs,
                   steps_per_epoch=steps_per_epoch,
                   validation_data=val_dataset,
                   validation_steps=validation_steps,
                   verbose=2)

Epoch 1/2


W0727 01:18:09.136595 25636 deprecation.py:323] From c:\users\jwlee\anaconda3\envs\tf2_gpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


581/581 - 182s - loss: 0.1761 - accuracy: 0.9732 - val_loss: 0.1122 - val_accuracy: 0.9831
Epoch 2/2
581/581 - 159s - loss: 0.1339 - accuracy: 0.9912 - val_loss: 0.1080 - val_accuracy: 0.9857
